In [1]:
#!/usr/bin/python3.8
import os
import json
import numpy as np
import pandas as pd
import easyvvuq as uq
from matplotlib import pyplot as plt

plt.rc('font', family='serif')

In [2]:
# Get the current working directory
cwd = os.getcwd()

# Load campaign
campaign = uq.Campaign(name='UQAAA_order2', \
    db_location='sqlite:///' + os.path.join(cwd, 'run/campaign.db'))

In [ ]:
# Check the list of runs
list_runs = campaign.list_runs()
with open('list_runs.json', 'w') as f:
    json.dump(list_runs, f, indent=2)

# Statistics of input parameters
params = list_runs[0][1]['params'].keys()
cv_params = np.array([])
for param in params:
    samples = np.array([])
    for i in range(len(list_runs)):
        samples = np.append(samples, list_runs[i][1]['params'][param])
    cv_params = np.append(cv_params, abs(samples.std()/samples.mean()))
    print(param, samples.mean(), samples.std(), abs(samples.std()/samples.mean()))
print('Mean coefficient of variation:', cv_params.mean())

In [3]:
ref_outlet = 2
paramSym = {
    'Lambda':'$\Lambda$',
    'E':'$E$',
    'F':'$F$',
    'Re':'$Re$',
    'Wo':'$Wo$',
    'm':'$m$',
    'gamma_R':'$\gamma_R$',
    'gamma_C':'$\gamma_C$',
    'shotShift': '$T_\min$'
}
qoi_cols = [
    'Qratios_Desired',
    'Qratios_Measured',
    'Qratios_RelErr',
    'TAWSS',
    'OSI',
    'ECAP',
    'RRT'
]

# Analyse the results of simulations
results = campaign.analyse(qoi_cols=qoi_cols)

In [ ]:
for root, dirs, files in os.walk(campaign.get_campaign_runs_dir()):
    for name in files:
        if name == 'riskFactors.csv':
            pathToGrid = os.path.join(root, name)
            break
grid = pd.read_csv(pathToGrid, usecols=['grid_x', 'grid_y', 'grid_z'])

rF_all = pd.DataFrame()
for i in range(1, campaign.get_active_sampler().n_samples + 1):
    file = campaign.get_campaign_runs_dir() + '/run_' + str(i) + '/periods_3/riskFactors.csv'
    rF = pd.read_csv(file, usecols=['TAWSS', 'OSI', 'RRT', 'ECAP'])
    rF_all = pd.concat([rF_all, rF])
rF_mean = rF_all.groupby(rF_all.index, as_index=False).mean()
rF_std = rF_all.groupby(rF_all.index, as_index=False).std()
rF_cv = rF_std / rF_mean
rF_mean.columns=['TAWSS_mean', 'OSI_mean', 'RRT_mean', 'ECAP_mean']
rF_std.columns=['TAWSS_std', 'OSI_std', 'RRT_std', 'ECAP_std']
rF_cv.columns=['TAWSS_cv', 'OSI_cv', 'RRT_cv', 'ECAP_cv']

riskFactors_overall = pd.concat([grid, rF_mean, rF_std, rF_cv], axis=1)
riskFactors_overall.to_csv('riskFactors_overall.csv', index=False)

In [ ]:
# Statistics of qoi
cv_qoi = np.array([])
cvr_qoi = np.array([])
for qoi in qoi_cols[3:7]:
    stat = results.describe(qoi)[qoi][0]
    cv = abs(stat.std()/stat.mean())
    cv_qoi = np.append(cv_qoi, cv)
    cvr_qoi = np.append(cvr_qoi, cv/cv_params.mean())
    print(qoi, stat.mean(), stat.std(), cv, cv/cv_params.mean())
print('Mean coefficient of variation:', cv_qoi.mean())
print('Mean coefficient of variation ratio:', cvr_qoi.mean())

In [ ]:
# Plot moments and Sobol indices of qoi
for i in range(0, 7):
    results.plot_moments(qoi=qoi_cols[i], filename='moments_' + qoi_cols[i] +'.pdf')
    plt.close()
    results.plot_sobols_first(qoi=qoi_cols[i], withdots=True, filename='sobolsFirst_' + qoi_cols[i] +'.pdf')
    plt.close()

param = 'Qratios_RelErr'
print('Mean\n', results.describe(param, 'mean'))
print('Standard deviation:\n', results.describe(param, 'std'))
print('First Sobol index:\n', results.sobols_first(param))
#print('Second Sobol index:\n', results.sobols_second(param))
print('Total Sobol index:\n', results.sobols_total(param))

In [ ]:
def CompareQratios(desired, measured, ref_outlet, filename):
    mean_desired = np.insert(desired.loc['mean'].to_numpy(), ref_outlet, 1)
    mean_measured = np.insert(measured.loc['mean'].to_numpy(), ref_outlet, 1)
    std_desired = np.insert(desired.loc['std'].to_numpy(), ref_outlet, 0)
    std_measured = np.insert(measured.loc['std'].to_numpy(), ref_outlet, 0)

    fig, ax = plt.subplots()
    mid = np.arange(len(mean_desired))
    ax.bar(mid - 0.2, mean_desired, color='tab:green', width=0.4, label='desired')
    ax.bar(mid + 0.2, mean_measured, color='tab:orange', width=0.4, label='measured')
    ax.errorbar(mid - 0.2, mean_desired, std_desired, ecolor='k', linestyle='None', \
                capsize=3, elinewidth=0.5)
    ax.errorbar(mid + 0.2, mean_measured, std_measured, ecolor='k', linestyle='None', \
                capsize=3, elinewidth=0.5)
    ax.set_xticks([i for i in range(10)])
    ax.legend()
    ax.grid(axis='y')
    ax.set_axisbelow(True)
    ax.minorticks_on()
    ax.tick_params(which='minor', bottom=False)
    ax.set_xlabel('Outlet index')
    ax.set_ylabel('Flow rate ratio')
    fig.savefig(filename, bbox_inches='tight')
    plt.close()

CompareQratios(
    results.describe('Qratios_Desired'), \
    results.describe('Qratios_Measured'), \
    ref_outlet, \
    'barChart_Qratios.pdf'
)

In [ ]:
def PlotRelativeErrors(relErr, ref_outlet, filename):
    mean_error = np.insert(relErr.loc['mean'].to_numpy(), ref_outlet, 0)
    std_error = np.insert(relErr.loc['std'].to_numpy(), ref_outlet, 0)

    fig, ax = plt.subplots()
    mid = np.arange(len(mean_error))
    ax.errorbar(mid, mean_error, std_error, color='k', linestyle='None', \
                capsize=5, elinewidth=1, marker='.', markersize=8)
    ax.grid(axis='y')
    ax.set_axisbelow(True)
    ax.minorticks_on()
    ax.tick_params(which='minor', bottom=False)
    ax.set_xticks([i for i in range(10)])
    ax.set_xlabel('Outlet index')
    ax.set_ylabel('Relative error')
    fig.savefig(filename, bbox_inches='tight')
    plt.close()

PlotRelativeErrors(results.describe('Qratios_RelErr'), ref_outlet, 'relErr_Qratios.pdf')
print('mean:', results.describe('Qratios_RelErr', 'mean'))
print('std:', results.describe('Qratios_RelErr', 'std'))

In [ ]:
from matplotlib import gridspec as gridspec
shade = {
    'TAWSS':[results.describe('TAWSS', 'mean')[2], 0.15],
    'OSI':[0.36, results.describe('OSI', 'mean')[6]],
    'ECAP':[2.4, results.describe('ECAP', 'mean')[6]],
    'RRT':[14.9, results.describe('RRT', 'mean')[5]]
}

def BoxWhiskerPlot(results, qoi_cols, moment, filename):
    fig, ax = plt.subplots(len(qoi_cols), 1)
    gs = gridspec.GridSpec(len(qoi_cols)*2,1)
    for i in range(len(qoi_cols)):
        stat = [
            results.describe(qoi_cols[i], moment)[2], # min
            results.describe(qoi_cols[i], moment)[3], # lower quartile
            results.describe(qoi_cols[i], moment)[4], # median
            results.describe(qoi_cols[i], moment)[5], # upper quartile
            results.describe(qoi_cols[i], moment)[6], # max
        ]
        ax[i].boxplot(stat, vert=False, widths=0.7, medianprops={'color':'black'}, whis=100, sym='')
        if moment == 'mean':
            ax[i].axvspan(shade[qoi_cols[i]][0], shade[qoi_cols[i]][1], color='tab:red', alpha=0.3)

        ax[i].set_position(gs[i*2, 0].get_position(fig))
        ax[i].tick_params(left=False)
        ax[i].minorticks_on()
        ax[i].spines['top'].set_visible(False)
        ax[i].spines['left'].set_visible(False)
        ax[i].spines['right'].set_visible(False)
        ax[i].set_yticklabels([qoi_cols[i]])
    fig.savefig(filename, bbox_inches='tight')
    plt.close()

BoxWhiskerPlot(results, qoi_cols[3:7], 'mean', 'boxWhisker_riskFactors_mean.pdf')
print('Mean')
for i in range(3, 7):
    print(qoi_cols[i], ':',
          'lower quartile =', results.describe(qoi_cols[i], 'mean')[3], ',',
          'upper quartile =', results.describe(qoi_cols[i], 'mean')[5], ',',
          'max =', results.describe(qoi_cols[i], 'mean')[6]
    )
print()
BoxWhiskerPlot(results, qoi_cols[3:7], 'std', 'boxWhisker_riskFactors_std.pdf')
print('Standard deviation')
for i in range(3, 7):
    print(qoi_cols[i], ':',
          'lower quartile =', results.describe(qoi_cols[i], 'std')[3], ',',
          'upper quartile =', results.describe(qoi_cols[i], 'std')[5], ',',
          'max =', results.describe(qoi_cols[i], 'std')[6]
    )

In [21]:
from matplotlib.pyplot import cm
from matplotlib import colors as mcolors
def PlotFirstOrTotalSobols_Qratios(results, ref_outlet, order, filename):
    color = cm.Set1.colors
    if order == 'First':
        sobols = results.sobols_first('Qratios_RelErr')
    elif order == 'Total':
        sobols = results.sobols_total('Qratios_RelErr')
    
    fig, ax = plt.subplots()
    outlets = [i for i in range(10)]
    outlets.pop(ref_outlet)
    bottom = np.zeros(9)
    i = 0
    for input, value in sobols.items():
        ax.bar(outlets, value, color=color[i], width=0.7, label=paramSym[input], bottom=bottom)
        bottom += value
        i = i + 1
    ax.legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncols=3)
    ax.grid(axis='y')
    ax.set_axisbelow(True)
    ax.minorticks_on()
    ax.tick_params(which='minor', bottom=False)
    ax.set_xticks([i for i in range(10)])
    ax.set_xlabel('Outlet index')
    ax.set_ylabel(order + ' Sobol\' index')
    fig.savefig(filename, bbox_inches='tight')
    plt.close()

def PlotFirstOrTotalSobols_RiskFactors(results, qoi_cols, order, filename):
    width = 0.8 / len(qoi_cols)
    color = [i for i in mcolors.TABLEAU_COLORS]
    fig, ax = plt.subplots()
    for i in range(len(qoi_cols)):
        if order == 'First':
            sobols = results.sobols_first(qoi_cols[i])
        elif order == 'Total':
            sobols = results.sobols_total(qoi_cols[i])
        j = 0
        for input in sobols:
            if j == 0:
                ax.bar(j + width * i - 0.4 + width / 2, \
                       sobols[input][0], color=color[i], width=width, label=qoi_cols[i])
            else:
                ax.bar(j + width * i - 0.4 + width / 2, \
                       sobols[input][0], color=color[i], width=width)
            j = j + 1
    ax.legend()
    ax.grid(axis='y')
    ax.set_axisbelow(True)
    ax.minorticks_on()
    ax.tick_params(which='minor', bottom=False)
    ax.set_xticks([i for i in range(len(sobols))])
    ax.set_xticklabels([paramSym[i] for i in sobols.keys()])
    ax.set_xlabel('Input parameters')
    ax.set_ylabel(order + ' Sobol\' index')
    fig.savefig(filename, bbox_inches='tight')
    plt.close()

PlotFirstOrTotalSobols_Qratios(results, ref_outlet, 'First', 'sobolFirst_Qratios.pdf')
PlotFirstOrTotalSobols_Qratios(results, ref_outlet, 'Total', 'sobolTotal_Qratios.pdf')
PlotFirstOrTotalSobols_RiskFactors(results, qoi_cols[3:7], 'First', 'sobolFirst_riskFactors.pdf')
PlotFirstOrTotalSobols_RiskFactors(results, qoi_cols[3:7], 'Total', 'sobolTotal_riskFactors.pdf')


In [ ]:
from matplotlib import ticker as ticker
from matplotlib import colormaps as mcmap
def PlotSecondSobols(results, qoi, filename):
    sobols_second = results.sobols_second(qoi)
    sobols = np.zeros([len(sobols_second), len(sobols_second)])
    i = 0
    for input_x in sobols_second:
        j = 0
        for input_y in sobols_second:
            if i == j:
                sobols[i][j] = None
            else:
                sobols[i][j] = sobols_second[input_x][input_y][0]
            j = j + 1
        i = i + 1

    fig, ax = plt.subplots()
    img = ax.imshow(sobols, cmap=mcmap['Reds'])
    fig.colorbar(img)

    ax.grid(which='minor', axis='both', color='k', linewidth=0.5)
    ax.minorticks_on()
    ax.xaxis.set_minor_locator(ticker.LinearLocator(numticks=len(sobols_second)+1))
    ax.yaxis.set_minor_locator(ticker.LinearLocator(numticks=len(sobols_second)+1))
    ax.set_xticks([i for i in range(len(sobols_second))])
    ax.set_xticklabels([paramSym[i] for i in sobols_second.keys()])
    ax.set_yticks([i for i in range(len(sobols_second))])
    ax.set_yticklabels([paramSym[i] for i in sobols_second.keys()])
    ax.tick_params(bottom=False, left=False)
    ax.set_xlabel('Input variables')
    ax.set_ylabel('Input variables')
    fig.savefig(filename, bbox_inches='tight')
    plt.close()

for i in range(3, 7):
    PlotSecondSobols(results, qoi_cols[i], 'sobolSecond_' + qoi_cols[i] + '.pdf')